In [160]:
from pyspark.sql.functions import regexp_replace ,count,mean,max,min,desc,asc,expr

In [2]:
from pyspark import SparkConf,SparkContext
sconf =SparkConf().setMaster("yarn").setAppName("shu")
sc=SparkContext(conf=sconf)
sc
import sys,os
os.environ["SPARK_HOME"]='/opt/cloudera/parcels/CDH/lib/spark'
sys.path.append('/opt/cloudera/parcels/CDH/lib/spark/python')
sys.path.append('/opt/cloudera/parcels/CDH/lib/spark/python/lin/py4j-0.9-src.zip')
import pyspark
import findspark
findspark.init()
### Initialize the spark
from pyspark.sql import SparkSession, regexp_replace 
spark = SparkSession.builder.getOrCreate()


In [4]:
bank =spark.read.format("csv").option("sep","\t").load("Project1.csv")

In [8]:
bd1=bank.withColumn("_c0",regexp_replace("_c0",'"',"")).rdd

In [10]:
bd1.take(2)

[Row(_c0='age;job;marital;education;default;balance;housing;loan;contact;day;month;duration;campaign;pdays;previous;poutcome;y'),
 Row(_c0='58;management;married;tertiary;no;2143;yes;no;unknown;5;may;261;1;-1;0;unknown;no')]

In [11]:
bd2=bd1.map(lambda x:x[0].split(";"))

In [12]:
header=bd2.first()

In [14]:
bd3=bd2.filter(lambda row:row!=header)

In [17]:
bank =bd3.map(lambda x:(int(x[0]),x[1],x[2],x[3],x[4],int(x[5]),x[6],x[7],x[8],int(x[9]),x[10],int(x[11]),int(x[12]),int(x[13]),int(x[14]),x[15],x[16]))

In [19]:
data=bank.toDF(header)

In [20]:
data.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [31]:
type(data)

pyspark.sql.dataframe.DataFrame

In [23]:
data.printSchema()

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: long (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



2.	Give marketing success rate (No. of people subscribed / total no. of entries)   
a.	Give marketing failure rate


In [47]:
(data.filter(data["y"]=="yes").count()/data.count())*100


11.698480458295547

In [49]:
(data.filter(data["y"]=="no").count()/data.count())*100

88.30151954170445

3.	Give the maximum, mean, and minimum age of the average targeted customer

In [52]:
data.select(max(data["age"]),mean(data["age"]),min(data["age"])).show()

+--------+-----------------+--------+
|max(age)|         avg(age)|min(age)|
+--------+-----------------+--------+
|      95|40.93621021432837|      18|
+--------+-----------------+--------+



4. Check the quality of customers by checking average balance, median balance of customers

In [146]:
data.select(mean(data["balance"])).show()

+------------------+
|      avg(balance)|
+------------------+
|1362.2720576850766|
+------------------+



5.	Check if age matters in marketing subscription for deposit

In [66]:
a=data.filter(data["y"]=="yes")

In [90]:
b=a.groupBy("age").count()

In [91]:
b.show()

+---+-----+
|age|count|
+---+-----+
| 29|  171|
| 26|  134|
| 65|   21|
| 54|   84|
| 19|   11|
| 22|   40|
| 77|   22|
| 34|  198|
| 50|   72|
| 57|   78|
| 32|  221|
| 43|  103|
| 84|    5|
| 31|  206|
| 39|  143|
| 25|  113|
| 95|    1|
| 71|   25|
| 68|   21|
| 72|   24|
+---+-----+
only showing top 20 rows



In [84]:
b.show()

+---+-----+
|age|count|
+---+-----+
| 29|  171|
| 26|  134|
| 65|   21|
| 54|   84|
| 19|   11|
| 22|   40|
| 77|   22|
| 34|  198|
| 50|   72|
| 57|   78|
| 32|  221|
| 43|  103|
| 84|    5|
| 31|  206|
| 39|  143|
| 25|  113|
| 95|    1|
| 71|   25|
| 68|   21|
| 72|   24|
+---+-----+
only showing top 20 rows



In [101]:
data.createOrReplaceTempView("bank")

In [105]:
spark.sql("select age,count(y) as Members from bank where y =='yes' group by age order by  Members desc").show()

+---+-------+
|age|Members|
+---+-------+
| 32|    221|
| 30|    217|
| 33|    210|
| 35|    209|
| 31|    206|
| 34|    198|
| 36|    195|
| 29|    171|
| 37|    170|
| 28|    162|
| 38|    144|
| 39|    143|
| 27|    141|
| 26|    134|
| 41|    120|
| 46|    118|
| 40|    116|
| 25|    113|
| 47|    113|
| 42|    111|
+---+-------+
only showing top 20 rows



In [108]:
spark.sql("select age,count(y) as Members from bank where y =='yes'  group by age having Members >'200' order by Members desc").show()

+---+-------+
|age|Members|
+---+-------+
| 32|    221|
| 30|    217|
| 33|    210|
| 35|    209|
| 31|    206|
+---+-------+



as you can see in the age of 30-35 there are members 

6.	Check if marital status mattered for a subscription to deposit

In [ ]:
as shown below it is observed that married person and single person are more inclined towards subsciption

In [112]:
spark.sql('select marital,count(y) as Members from bank where y =="yes" group by marital order by Members').show()

+--------+-------+
| marital|Members|
+--------+-------+
|divorced|    622|
|  single|   1912|
| married|   2755|
+--------+-------+



7.	Check if age and marital status together mattered for a subscription to deposit scheme

In [117]:
spark.sql('select age,marital,count(y) as Members from bank where y =="yes" group by age, marital order by Members desc').show()

+---+-------+-------+
|age|marital|Members|
+---+-------+-------+
| 30| single|    151|
| 28| single|    138|
| 29| single|    133|
| 32| single|    124|
| 26| single|    121|
| 34|married|    118|
| 31| single|    111|
| 27| single|    110|
| 35|married|    101|
| 36|married|    100|
| 25| single|     99|
| 37|married|     98|
| 33|married|     97|
| 33| single|     97|
| 32|married|     87|
| 39|married|     87|
| 38|married|     86|
| 35| single|     84|
| 47|married|     83|
| 31|married|     80|
+---+-------+-------+
only showing top 20 rows



yes as 30-35 age range and marital status should be single or married

8.	Do feature engineering for the bank and find the right age effect on the campaign.

for feature engineering as age is one of best variable that has good weight on the result so we convert this into categorical variable

In [164]:
data = data.withColumn("age_group", expr("CASE WHEN age <20 THEN 'teen_age' " + 
               "WHEN age >=20 and age<40 THEN 'middle_aged' " +
               "ELSE 'old_aged' END"))

In [166]:
data.createOrReplaceTempView("Newbank")

In [167]:
from pyspark.ml.feature import StringIndexer

In [176]:
a=StringIndexer(inputCol='age_group',outputCol='age_group2')


In [177]:
data = a.fit(data).transform(data)

In [181]:
data.createOrReplaceTempView("p")


In [183]:
spark.sql("select age_group, max(age_group2) as Preference from P group by age_group order by Preference asc").show()

+-----------+----------+
|  age_group|Preference|
+-----------+----------+
|middle_aged|       0.0|
|   old_aged|       1.0|
|   teen_age|       2.0|
+-----------+----------+



here 0 is given to the most prefered age category that is Middle Age(20,40)